In [6]:
import pickle
import numpy as np

In [58]:
%store -r X_test

X_test.groupby(['label']).count().iloc[:, 0]

label
0    15
1    12
2    18
3    20
5     4
6    20
7     9
8    14
9    16
Name: x, dtype: int64

In [49]:
model_0 = pickle.load(open('models/hmm_model_1.pkl', 'rb'))

for 

angles_0 = test['angles'].iloc[2]
angles_0 = [[one] for one in angles_0]
angles_0

model_0.score_samples(angles_0)

(8.414020726733833,
 array([[4.50947293e-95, 1.24864833e-02, 1.06679199e-03, 9.86446367e-01,
         2.28278971e-09, 3.55089197e-07],
        [7.55873105e-38, 1.03982244e-02, 5.51445249e-05, 9.89546625e-01,
         4.76694940e-11, 6.01148298e-09],
        [1.83691946e-37, 1.27544613e-02, 8.15896677e-06, 9.87237374e-01,
         2.41843487e-10, 5.10929605e-09],
        [4.09521052e-35, 2.33729439e-02, 1.04812150e-05, 9.76616547e-01,
         2.25710387e-08, 5.14917287e-09],
        [4.50908724e-34, 6.09860832e-02, 2.96852836e-05, 9.38981419e-01,
         2.80497277e-06, 7.58797591e-09],
        [5.20684191e-32, 1.52545168e-01, 1.33797407e-04, 8.47088423e-01,
         2.32391055e-04, 2.20328408e-07]]))